# Vidio Sentiment Analyzer : Unveiling User Sentiments in Google Play Reviews

`Vidio Sentiment Analyzer: Mengungkap Sentimen Pengguna dalam Ulasan Google Play` merupakan pendekatan inovatif untuk mengidentifikasi pola-pola utama dalam kumpulan ulasan pengguna. Di era informasi digital yang berkembang pesat seperti sekarang, sangat penting bagi Vidio sebagai platform terkemuka untuk mengetahui dan memahami isu-isu utama yang diungkapkan oleh pengguna.

`Sentiment Analyzer` menggunakan teknik canggih untuk memahami sentimen yang mendominasi dalam ulasan Google Play. Dengan volume feedback pengguna yang besar, menjadi sangat penting bagi Vidio untuk mendapatkan wawasan tentang sentimen pengguna.

Penerapan analisis sentimen memungkinkan Vidio mengkategorikan ulasan pengguna menjadi sentimen positif dan negatif memungkinkan pemahaman yang lebih mendalam tentang pengalaman dan preferensi pengguna. Metrik evaluasi yang relevan dapat mencakup distribusi polaritas sentimen, intensitas sentimen, dan tren sentimen dari waktu ke waktu.

Dalam konteks ini, metrik evaluasi kunci dapat melibatkan metrik-metrik standar analisis sentimen seperti akurasi, precision, recall, dan F1 Score. Metrik-metrik ini memberikan pandangan komprehensif tentang seberapa baik Sentiment Analyzer dalam mengklasifikasikan sentimen pengguna dengan akurat.

Wawasan yang diperoleh dari Sentiment Analyzer dapat memberdayakan Vidio untuk meningkatkan kepuasan pengguna, memprioritaskan perbaikan fitur, dan menanggapi masalah dengan cepat. Dengan fokus pada sentimen pengguna, Vidio dapat secara strategis menyelaraskan upayanya untuk memenuhi harapan pengguna dan menjaga pengalaman pengguna yang positif.

Melalui Vidio Sentiment Analyzer, Vidio dapat menavigasi lanskap dinamis opini pengguna di platform Google Play, memungkinkan pengambilan keputusan yang berbasis informasi dan langkah-langkah proaktif untuk meningkatkan kepuasan pengguna.

# Data Preparation

## Import Library

In [2]:
import pandas as pd
from datetime import datetime
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from gensim import corpora, models
import pyLDAvis.gensim_models as gensimvis
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.tokenize import word_tokenize
import pycrfsuite 
# Download data NLTK
nltk.download('stopwords')
nltk.download('wordnet')
import re
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from nltk.tag import CRFTagger
from nltk.stem import PorterStemmer
from collections import Counter
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from gensim.corpora.dictionary import Dictionary
from gensim.models import LdaMulticore
from gensim.models import LdaModel
from gensim.models.coherencemodel import CoherenceModel
from pprint import pprint

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## Data Gathering

Data yang digunakan merupakan data yang diambil atau discrapping dari google play pada aplikasi Vidio mulai tanggal 2020-08-03 - 2023-12-20 sebanyak 10000 rows

In [3]:
# Read Data
data = pd.read_csv('dataset/data_vidio.csv')
# sanity check
data

,content,score,at,sentiment
0,"Sekarang banyak iklan nya, nonton sinetron, dr...",1,2023-12-20 03:03:49,Negatif
1,Vidio lagi knp sih?? Tayangan EPL nya loading ...,3,2023-12-10 23:45:09,Negatif
2,Kenapa sudah berlangganan tapi masih muncul ik...,2,2023-12-10 21:01:34,Negatif
3,"Kualitas video nya jangan otomatis dong,kalau ...",1,2023-12-07 17:42:09,Negatif
4,"Kenapa ya sy mau beli paketnya ga bisa2, alesa...",1,2023-12-12 22:05:49,Negatif
...,...,...,...,...
9995,Agak lemot sih tapi bagus kok kalau mati lampu...,5,2018-12-05 11:38:33,Positif
9996,Aplikasi nggk guna buang2 kuota aj kirain grat...,1,2020-04-15 15:52:46,Negatif
9997,Saya suka banget sama aplikasi Vidio ini karen...,4,2020-10-17 20:16:41,Positif
9998,Sangat baguss tapi kenapa yah kalau buka video...,5,2018-10-27 17:39:24,Positif


## Data Definition

Data yang digunakan merupakan data yang diambil atau discrap dari google play review aplikasi vidio dengan jumlah data sebesar 10000 rows dan 3 kolom, berikut definisi masing masing kolomnya

- `content`: content merupakan komentar atau review yang diberikan para user untuk aplikasi vidio di google play store
- `score`: score merupakan penilaian dari user dengan skala 1-5, semakin tinggi nilainya maka akan semakin reviewnya, dan semakin rendah maka akan semakin buruk reviewnya.
- `at`: at merupakan tanggal saat user melakukan atau mengirim review di google play store
- `sentiment`: sentiment merupakan akumulasi dari score, jika score memiliki nilai dibawah atau sama dengan tiga maka dapat disimpulkan sentimennya negatif dan jika score memiliki nilai diatas 3 maka dapat disimpulkan sentimentnya positif

## Data Validation

Proses mengevaluasi dan memvalidasi integritas, kualitas, dan kecocokan data yang digunakan dalam proses pembangunan dan evaluasi model machine learning. Hal ini bertujuan untuk memastikan bahwa data yang digunakan memenuhi persyaratan dan standar yang diperlukan untuk menghasilkan model yang akurat dan dapat diandalkan.

### Cek Informasi Data

In [4]:
# Check informasi data
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   content    10000 non-null  object
 1   score      10000 non-null  int64 
 2   at         10000 non-null  object
 3   sentiment  10000 non-null  object
dtypes: int64(1), object(3)
memory usage: 312.6+ KB


In [5]:
# Mengubah tipe data kolom "at" menjadi datetime dengan format yang diinginkan
data['at'] = pd.to_datetime(data['at'], format='%Y-%m-%d %H:%M:%S')

In [6]:
# Ekstraksi tanggal, jam, dan menit
data['event_date'] = data['at'].dt.date
data['event_hour'] = data['at'].dt.hour
data['event_minute'] = data['at'].dt.minute

In [7]:
# sanityy check
data

,content,score,at,sentiment,event_date,event_hour,event_minute
0,"Sekarang banyak iklan nya, nonton sinetron, dr...",1,2023-12-20 03:03:49,Negatif,2023-12-20,3,3
1,Vidio lagi knp sih?? Tayangan EPL nya loading ...,3,2023-12-10 23:45:09,Negatif,2023-12-10,23,45
2,Kenapa sudah berlangganan tapi masih muncul ik...,2,2023-12-10 21:01:34,Negatif,2023-12-10,21,1
3,"Kualitas video nya jangan otomatis dong,kalau ...",1,2023-12-07 17:42:09,Negatif,2023-12-07,17,42
4,"Kenapa ya sy mau beli paketnya ga bisa2, alesa...",1,2023-12-12 22:05:49,Negatif,2023-12-12,22,5
...,...,...,...,...,...,...,...
9995,Agak lemot sih tapi bagus kok kalau mati lampu...,5,2018-12-05 11:38:33,Positif,2018-12-05,11,38
9996,Aplikasi nggk guna buang2 kuota aj kirain grat...,1,2020-04-15 15:52:46,Negatif,2020-04-15,15,52
9997,Saya suka banget sama aplikasi Vidio ini karen...,4,2020-10-17 20:16:41,Positif,2020-10-17,20,16
9998,Sangat baguss tapi kenapa yah kalau buka video...,5,2018-10-27 17:39:24,Positif,2018-10-27,17,39


In [8]:
# sanity check
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   content       10000 non-null  object        
 1   score         10000 non-null  int64         
 2   at            10000 non-null  datetime64[ns]
 3   sentiment     10000 non-null  object        
 4   event_date    10000 non-null  object        
 5   event_hour    10000 non-null  int32         
 6   event_minute  10000 non-null  int32         
dtypes: datetime64[ns](1), int32(2), int64(1), object(3)
memory usage: 468.9+ KB


In [9]:
data['event_date'] = pd.to_datetime(data['event_date'])

In [10]:
data

,content,score,at,sentiment,event_date,event_hour,event_minute
0,"Sekarang banyak iklan nya, nonton sinetron, dr...",1,2023-12-20 03:03:49,Negatif,2023-12-20,3,3
1,Vidio lagi knp sih?? Tayangan EPL nya loading ...,3,2023-12-10 23:45:09,Negatif,2023-12-10,23,45
2,Kenapa sudah berlangganan tapi masih muncul ik...,2,2023-12-10 21:01:34,Negatif,2023-12-10,21,1
3,"Kualitas video nya jangan otomatis dong,kalau ...",1,2023-12-07 17:42:09,Negatif,2023-12-07,17,42
4,"Kenapa ya sy mau beli paketnya ga bisa2, alesa...",1,2023-12-12 22:05:49,Negatif,2023-12-12,22,5
...,...,...,...,...,...,...,...
9995,Agak lemot sih tapi bagus kok kalau mati lampu...,5,2018-12-05 11:38:33,Positif,2018-12-05,11,38
9996,Aplikasi nggk guna buang2 kuota aj kirain grat...,1,2020-04-15 15:52:46,Negatif,2020-04-15,15,52
9997,Saya suka banget sama aplikasi Vidio ini karen...,4,2020-10-17 20:16:41,Positif,2020-10-17,20,16
9998,Sangat baguss tapi kenapa yah kalau buka video...,5,2018-10-27 17:39:24,Positif,2018-10-27,17,39


In [11]:
# sanity check
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   content       10000 non-null  object        
 1   score         10000 non-null  int64         
 2   at            10000 non-null  datetime64[ns]
 3   sentiment     10000 non-null  object        
 4   event_date    10000 non-null  datetime64[ns]
 5   event_hour    10000 non-null  int32         
 6   event_minute  10000 non-null  int32         
dtypes: datetime64[ns](2), int32(2), int64(1), object(2)
memory usage: 468.9+ KB


In [12]:
data = data.sort_values(by='event_date')

In [13]:
data

,content,score,at,sentiment,event_date,event_hour,event_minute
5575,Sumpah aplikasi ini bikin gw kesel sendiri. Ud...,1,2018-09-15 08:17:04,Negatif,2018-09-15,8,17
8625,lumayan bagus tapi tolong di perbaiki gangguan...,5,2018-09-15 15:50:21,Positif,2018-09-15,15,50
9029,Bagus banget Jadi bisa lihat TV dimn pun saat ...,5,2018-09-15 14:33:19,Positif,2018-09-15,14,33
6474,"Masa tidak bisa tampilkan semua liga, kami mau...",1,2018-09-19 02:10:42,Negatif,2018-09-19,2,10
6954,"Kayaknya aplikasi OONA lebih bagus, bagi yg ke...",2,2018-09-20 09:13:09,Negatif,2018-09-20,9,13
...,...,...,...,...,...,...,...
56,Aplikasi dari 2015 tapi film dan anime yang te...,1,2023-12-29 04:32:05,Negatif,2023-12-29,4,32
626,Saya menyesal berlanggan vidio karena pembayar...,1,2023-12-29 20:19:22,Negatif,2023-12-29,20,19
1851,tolong apknya di update dong masa gak ada film...,1,2023-12-29 13:37:03,Negatif,2023-12-29,13,37
59,masa nonton premier league vidionya lebih lamb...,1,2023-12-29 18:35:41,Negatif,2023-12-29,18,35


### Check Nilai Null
Mengecek apakah data terdapat nilai null pada setiap kolom

In [14]:
#Menghitung nilai null pada kolom
data.isnull().sum()

content         0
score           0
at              0
sentiment       0
event_date      0
event_hour      0
event_minute    0
dtype: int64

### Melihat statistik deskriptif dan Range Data Numerikal

Cek statistika deskriptif

In [15]:
#Melihat statistika deskriptif dengan describe()
data.describe()

,score,at,event_date,event_hour,event_minute
count,10000.000000,10000,10000,10000.000000,10000.000000
mean,2.277700,2022-05-10 04:13:30.026399744,2022-05-09 13:16:45.120000256,14.446700,29.457400
min,1.000000,2018-09-15 08:17:04,2018-09-15 00:00:00,0.000000,0.000000
25%,1.000000,2021-03-21 17:04:09.750000128,2021-03-21 00:00:00,10.000000,14.000000
50%,1.000000,2022-09-15 18:43:19.500000,2022-09-15 00:00:00,16.000000,30.000000
75%,4.000000,2023-08-26 21:09:23.750000128,2023-08-26 00:00:00,20.000000,44.000000
max,5.000000,2023-12-29 20:23:59,2023-12-29 00:00:00,23.000000,59.000000
std,1.564745,NaN,NaN,6.758855,17.342516


### Melihat Dimensi Data

In [16]:
data.shape

(10000, 7)

In [17]:
data.sentiment.value_counts()

sentiment
Negatif    7461
Positif    2539
Name: count, dtype: int64

## Save Data

In [18]:
data.to_csv("dataset/data_preparation_vidio.csv", index = False)